In [ ]:
# Standard library
from os.path import join

# Third-party
import corner
import astropy.time as atime
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.celestialmechanics import OrbitalParams
from thejoker.plot import plot_rv_curves, _prev_result_color

plt.style.use('../thejoker/thejoker.mplstyle')

from figurehelpers import samples_units, make_rv_curve_figure, apw_corner

In [ ]:
dpi = 256

In [ ]:
# experiment number
e_number = 4
e_name = 'apogee'

In [ ]:
data_filename = "../data/troup-allVisit.h5"

## First read the orbital parameters sampled for this experiment:

In [ ]:
hdf5_key = name = '2M00110648+6609349' # HACK: This is hard coded and BAD!!
print(name)

In [ ]:
# read the data
with h5py.File(data_filename, 'r') as f:
    data = RVData.from_hdf5(f[hdf5_key])

In [ ]:
# read the samples from fixing the jitter
with h5py.File(join(paths.cache, 'experiment{}-fixed-jitter.h5'.format(e_number)), 'r') as g:
    pars1 = OrbitalParams.from_hdf5(g)
    samples1 = pars1.pack(plot_transform=True, units=samples_units)
    samples1[:,3] = (samples1[:,3] - 360*((data.t_offset/pars1._P) % 1.)) % (360) # HACK
    
# read the samples from sampling over the jitter
with h5py.File(join(paths.cache, 'experiment{}-sample-jitter.h5'.format(e_number)), 'r') as g:
    pars2 = OrbitalParams.from_hdf5(g)
    samples2 = pars2.pack(plot_transform=True, units=samples_units)
    samples2[:,3] = (samples2[:,3] - 360*((data.t_offset/pars2._P) % 1.)) % (360) # HACK

print("{} samples survived (a)".format(samples1.shape[0]))
print("{} samples survived (b)".format(samples2.shape[0]))

### Read in Troup's fit values for the orbital parameters:

In [ ]:
all_troup_data = np.genfromtxt(join(paths.root, "data", "troup16-dr12.csv"),
                               delimiter=",", names=True, dtype=None)
troup_data = all_troup_data[all_troup_data['APOGEE_ID'].astype(str) == name]

In [ ]:
troup_P = troup_data['PERIOD'] * u.day
troup_ecc = troup_data['ECC']
troup_K = troup_data['SEMIAMP']*u.m/u.s
troup_omega = (troup_data['OMEGA'] % (2*np.pi)) * u.radian

dt0 = atime.Time(troup_data['TPERI'], format='jd', scale='tcb').tcb.mjd# - data.t_offset
troup_phi0 = ((2*np.pi*dt0/troup_P.value) % (2*np.pi)) * u.radian

troup_v0 = troup_data['V0'] * u.m/u.s
print(troup_data['SLOPE'])

_t0 = atime.Time(troup_data['T0'], format='jd', scale='tcb').tcb.mjd
derp = -((2*np.pi*_t0/troup_P.value) % (2*np.pi)) * u.radian

truth_pars = OrbitalParams(P=troup_P, K=troup_K, ecc=troup_ecc, omega=troup_omega, 
                           phi0=troup_phi0 + derp, v0=troup_v0)
truth_vec = truth_pars.pack(plot_transform=True, units=samples_units)[0]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

bmjd = data.t.tcb.mjd
dmjd = bmjd.max() - bmjd.min()
t_grid = np.linspace(bmjd.min() -0.25*dmjd, 
                     bmjd.max() + 0.25*dmjd, 
                     1024)

rv_unit = u.km/u.s

plot_rv_curves(pars2[:256], t_grid, rv_unit=rv_unit,
               data=data, ax=ax, 
               plot_kwargs={'color': '#aaaaaa', 'zorder': -100, 'marker': ''},
               data_plot_kwargs={'ecolor': 'k', 'markersize': 3, 'elinewidth': 1, 'alpha': 1., 'zorder': 100})

# plt.plot(t_grid, truth_pars.rv_orbit(0)(t_grid), marker='')

ax.set_rasterization_zorder(-1)
ax.set_ylim(-40, -18)

ax.set_title("Experiment {}: {}".format(e_number, name), fontsize=24)
fig.tight_layout()
fig.savefig(join(paths.figures, '{}-rv-curves.pdf'.format(e_name)), dpi=dpi)

## Corner plot

In [ ]:
# Make a corner plot of all samples
_med_v0 = np.median(samples2[:,-1])
_mad_v0 = np.median(np.abs(samples2[:,-1] - _med_v0))
ranges = [(4.8, 7.8), (-0.025, 1.025), (0,360), (0,360), (2.25, 7.25),
          (3.5, 10), (_med_v0 - 5*_mad_v0, _med_v0 + 5*_mad_v0)]
labels = OrbitalParams.get_labels(samples_units)

corner_style = dict(alpha=0.25, markersize=1., bins=16, color='#555555')

In [ ]:
fig = apw_corner(samples2, range=ranges, truths=truth_vec, labels=labels, truth_color=_prev_result_color, 
                 **corner_style)
fig.subplots_adjust(left=0.08, bottom=0.08)
fig.suptitle("Experiment {}".format(e_number), fontsize=24)
fig.savefig(join(paths.figures, '{}-corner.pdf'.format(e_name)), dpi=128)

In [ ]:
np.median(np.exp(samples2[:,4]))

In [ ]:
plt.hist(np.exp(samples2[:,4]))
plt.axvline(np.median(data.stddev.to(u.m/u.s)).value, color='r')